<center><h1>MAPD-B - Preprocessing of SEVN data for binary black holes mass distribution analysis</center></h1>

<center><i>Tommaso Bertola, Giacomo Di Prima, Giuseppe Viterbo, Marco Zenari</i></center>

# Introduction: the computational problem

Our aim for this project is to preprocess the data of multiple SEVN simulations of binary systems.

SEVN is Python program developed by the Astronomy Department at the University of Padua to simulate the evolution of binary systems. The evolution takes into account different physical phenomena which obey the patterns and laws obeserved in the Universe, especially those seen in the stellar tracks.

We will focus our attention specifically to those systems evolving into binary black holes. To study these systems we therefore need to extract from the whole dataset produced by SEVN only some information regarding the initial and final conditions of the binary system evolution.

The final goal is to obtain a simple and handy DataFrame listing only those features.

# Data structure

To better understading the problem, we will briefely describe the dataset we are given.

The dataset consists of a number of folders named after some "hyperparameters" given to SEVN while performing the simulations.
In our case, we are given 60 different folders, whose names are like `sevn_output_Z0.001A1L1`, `sevn_output_Z0.03A5L1`, ...

Inside each folder there are three kinds of files:
* output_{nthread}.csv
* logfile_{nthread}.dat
* evolved_{nthread}.dat

where {nthread} is a number ranging from 0 to 29, corresponding to the thread responsible for the computation of those simulations. 

On average, each of the `output_{nthread}.csv` files occupy 750MB, `logfile_{nthread}.dat` 200MB, and `evolved_{nthread}.dat` 50MB. 

In total, each folder occupies between 26 to 31GB of data for a gross total of around 1.7TB.


## File structure
#### output_{}.csv

#### logfile_{}.dat

#### evolved_{}.dat

# Tools

## Cluster configuration
bhbh-*

### Dask on top of the cluster

## NFS data sharing

# A single simple calculation
just the calculation we need to perform to the files. can be an abstract description (parsing data, joining,, concatenating, expected output)

# Our attempts
Here we show the code to 

Load the whole thing at once

FG_new

FG_normal

FG_bag

Brute force approach